### TwoAgenChat with coding

This notebook shows how to use a `coder` agent and a `runner` agent to implement a back-and-force coding-debuging conversation.

### Step 1: Install dependencies

In [1]:
#i "nuget:https://www.myget.org/F/agentchat/api/v3/index.json"

#r "nuget:AgentChat.Core,*-*"
#r "nuget:AgentChat.OpenAI,*-*"
#r "nuget:AgentChat.Example.Share,*-*"
// this package provides FunctionDefinition for `dotnet-interactive` command
#r "nuget:AgentChat.DotnetInteractiveFunction,*-*"

Restore sources https://www.myget.org/F/agentchat/api/v3/index.json Installed Packages AgentChat.Core, 0.3.1 AgentChat.DotnetInteractiveFunction, 0.3.1 AgentChat.Example.Share, 0.3.1 AgentChat.OpenAI, 0.3.1

### Step 2: Create a Coding Agent
Then we are going to create a `GPTAgent` that will be used to write dotnet script code to resolve the user's problem.

In [2]:
using AgentChat.Example.Share;
using AgentChat;
using System.Text.Json;
using Azure.AI.OpenAI;

var completeFunction = new FunctionDefinition{
    Name = "TaskComplete",
    Description = "task complete",
    Parameters = BinaryData.FromObjectAsJson(new
    {
        Type = "object",
        Properties = new
		{
		    msg = new
		    {
			    Type = @"string",
			    Description = @"msg",
		    },
        },
        Required = new []
		{
		    "msg",
		},
    },
    new JsonSerializerOptions
	{
		PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
	}),
};

var coder = Constant.GPT35.CreateAgent(
    name: "Coder",
    roleInformation: @"You act as dotnet coder, you write dotnet script to resolve task.
-workflow-
write code

if code_has_error
    fix_code_error

if task_complete, call TaskComplete

-end-

Here're some rules to follow on write_code_to_resolve_current_step:
- put code between ```csharp and ```
- Use top-level statements, remove main function, just write code, like what python does.
- Remove all `using` statement. Runner can't handle it.
- Try to use `var` instead of explicit type.
- Try avoid using external library.
- Don't use external data source, like file, database, etc. Create a dummy dataset if you need.
- Always print out the result to console. Don't write code that doesn't print out anything.

Here are some examples for write code:
```nuget
xxx
```
```csharp
xxx
```

Here are some examples for fix_code_error:
The error is caused by xxx. Here's the fix code
```csharp
xxx
```",
    temperature: 0,
    functionMap: new Dictionary<FunctionDefinition, Func<string, Task<string>>> {
        { completeFunction, async (args) => "[COMPLETE]"}
    });

### Step 3: Create a runner agent to run code and get output
In this step, we are going to create another `GPTAgent` that will be used to run the code and get the output.
The runner agent will use `function_call` to call `RunDotnetCodeFunction` and `InstallNugetDependencyFunction` to run the code and get the output.

In [3]:
using System.IO;
using AgentChat.DotnetInteractiveService;
using System.Text.Json;

var workDir = Path.Combine(Path.GetTempPath(), "InteractiveService");
if (!Directory.Exists(workDir))
    Directory.CreateDirectory(workDir);

var functionDefinition = new FunctionDefinition{
    Name = "Greeting",
    Description = "Greeting",
    Parameters = BinaryData.FromObjectAsJson(new
    {
        Type = "object",
        Properties = new
		{
		    greeting = new
		    {
			    Type = @"string",
			    Description = @"greeting",
		    },
        },
        Required = new []
		{
		    "greeting",
		},
    },
    new JsonSerializerOptions
	{
		PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
	}),
};

var service = new InteractiveService(workDir);
var dotnetInteractiveFunctions = new DotnetInteractiveFunction(service);

// this function is used to fix invalid json returned by GPT-3
var fixInvalidJsonFunction = new FixInvalidJsonFunctionWrapper(Constant.GPT35);

var runner = Constant.GPT35.CreateAgent(
    name: "Runner",
    roleInformation: @"you act as dotnet runner, you run dotnet script and install nuget packages. Here's the workflow you follow:
-workflow-
if code_is_available
    call run_code

if nuget_packages_is_available
    call install_nuget_packages

for any other case
    call greeting
-end-",
    temperature: 0,
    functionMap: new Dictionary<FunctionDefinition, Func<string, Task<string>>> {
        { dotnetInteractiveFunctions.RunCodeFunction, fixInvalidJsonFunction.FixInvalidJsonWrapper(dotnetInteractiveFunctions.RunCodeWrapper) },
        { dotnetInteractiveFunctions.InstallNugetPackagesFunction, dotnetInteractiveFunctions.InstallNugetPackagesWrapper },
        { functionDefinition, async (args) => "NO_CODE_AVAILABLE"}
    });

// start kenel
await service.StartAsync(workDir, default);

### Play with runner agent
Now we can play with the runner agent to see how it works.

In [4]:
// runner should respond with NO_CODE_AVAILABLE
var msg = await runner.SendMessageAsync("hello");
msg.PrettyPrintMessage();

// runner should run code
msg = await runner.SendMessageAsync("```csharp\nConsole.WriteLine(1+1+1);\n```");
msg.PrettyPrintMessage();

// runner should install nuget packages
msg = await runner.SendMessageAsync("```nuget\nMicrosoft.ML\n```");
msg.PrettyPrintMessage();

Message from Runner
--------------------
NO_CODE_AVAILABLE
--------------------

Message from Runner
--------------------
3

--------------------

Message from Runner
--------------------
Installed nuget packages:
- Microsoft.ML

--------------------



### Step 4: Create two agents chat

In this step, we are going to create a `TwoAgentChat` to let the `user` agent and the `coder` agent chat with each other.

The Coding agent will take the user's input and write dotnet script code to resolve the user's problem.

And once the code is ready, the Coding agent will send the code to the User agent.

Then the User agent will send the code to the Runner agent to run the code and get the output, then send the output back to the Coder agent.

If the output is correct, the Coder agent will return a 'COMPLETE' message back to User agent and wait for next input. Otherwise, the Coder agent will try to fix the code and send the code back to the User agent to run again.


In [6]:
// use runner agent to auto-reply message from coder
var user = runner.CreateAutoReplyAgent("User", async (msgs, ct) => {
    // if last message contains "COMPLETE", stop sending messages to runner agent and fall back to user agent
    if (msgs.Last().Content.Contains("[COMPLETE]"))
        return new Message(Role.Assistant, IChatMessageExtension.TERMINATE, from: "User"); // return TERMINATE to stop conversation
    
    // otherwise, send message to runner agent to either run code or install nuget packages and get the reply
    return await runner.SendMessageAsync(msgs.Last());
});

await user.SendMessageToAgentAsync(
    coder,
    "what's the 10th of fibonacci? Print the question and result in the end.",
    maxRound: 10);

Message from Coder
--------------------
```csharp
using System;

namespace Fibonacci
{
    class Program
    {
        static void Main(string[] args)
        {
            int n = 10;
            int result = Fibonacci(n);
            Console.WriteLine($"The 10th Fibonacci number is: {result}");
        }

        static int Fibonacci(int n)
        {
            if (n <= 1)
                return n;

            int[] fib = new int[n + 1];
            fib[0] = 0;
            fib[1] = 1;

            for (int i = 2; i <= n; i++)
            {
                fib[i] = fib[i - 1] + fib[i - 2];
            }

            return fib[n];
        }
    }
}
```

The 10th Fibonacci number is: 55
--------------------

Message from Runner
--------------------
Error: (3,1): error CS7021: Cannot declare namespace in script code
--------------------

Message from Coder
--------------------
The error is caused by declaring a namespace in the script code. In dotnet script, we don't need to declare a